# 🌻🌳 Árvore de Decisão - ENTROPIA 🌳🌻

## Importações

In [31]:
import pandas as pd
from sklearn.metrics import confusion_matrix, accuracy_score
from sklearn.metrics import classification_report
import graphviz
from sklearn.model_selection import train_test_split 
from sklearn import tree
from sklearn.metrics import multilabel_confusion_matrix
from sklearn.model_selection import GridSearchCV
from sklearn.compose import ColumnTransformer, make_column_selector
from sklearn.preprocessing import OrdinalEncoder,LabelEncoder

## Leitura da base

In [32]:
df = pd.read_excel('../../base/Incluses.xlsx')

## Seleção de Features

In [33]:
# Excluindo colunas inúteis para classificação do modelo
df = df.drop(columns=['Id', 'Hora de início','Hora de conclusão','Email','Nome','Coluna1'])

# Filtrando dados úteis
df = df[df['Você participa da comunidade LGBTQIA+'] == 'Sim']

# Mostrando df
display(df.head(5))

,Você participa da comunidade LGBTQIA+,Idade,Gênero,Orientação Sexual,Localização,Nível de Escolaridade,Você usa aplicativos para procurar oportunidades de emprego?,Preferência por Cursos,Você já sentiu desafios para entrar no mercado de trabalho devido ao seu gênero,Você tem interesse em empreender?,Qual é a sua situação diante ao mercado de trabalho?,"Você usa redes sociais (exemplo: X, Facebook, Instagram, Youtube)",Com quais finalidades você usaria o Aplicativo Incluses
2,Sim,Menos de 18 anos,Mulher cisgênero,Homossexual,Região Sudeste,Ensino Fundamental completo,Sim,Prefiro cursos online,Sim,Não,Empregado (a),Sim,Todos acima
3,Sim,Menos de 18 anos,Mulher cisgênero,Homossexual,Região Sudeste,Ensino Fundamental completo,Tenho interesse,"Não tenho preferência, pode ser online ou pres...",Sim,Não,Empregado (a),Sim,Não usaria o app
4,Sim,Menos de 18 anos,Mulher cisgênero,Pansexual,Região Norte,Ensino Fundamental completo,Não,"Não tenho preferência, pode ser online ou pres...",Não,Talvez,Empregado (a),Sim,Fazer cursos de qualificação
5,Sim,Menos de 18 anos,Mulher cisgênero,Bissexual,Região Sudeste,Ensino Fundamental completo,Não,"Não tenho preferência, pode ser online ou pres...",Sim,Não,Empregado (a),Sim,Não usaria o app
14,Sim,Menos de 18 anos,Homem transgênero,Prefiro não informar,Região Sudeste,Nenhum,Não,"Não tenho preferência, pode ser on-line ou pre...",Sim,Sim,Desempregado (a),Sim,Fazer cursos de qualificação


## Separando atributos da resposta

In [34]:
df_resposta = df['Com quais finalidades você usaria o Aplicativo Incluses']
df_atributos = df.iloc[:, :-1]

## Transformando dados qualitativos em dados quantitativos

In [35]:
#Treino

# Configurando pré-processador
preprocessador = ColumnTransformer(
    transformers=[
        ('ordinal_encoder', OrdinalEncoder(), make_column_selector(dtype_include=['object','bool'])),
    ],
    remainder='passthrough',
    verbose_feature_names_out= False 
)

# Transformando rótulos da variável resposta em dados qualitativos 
df_atributos_pre = pd.DataFrame(preprocessador.fit_transform(df_atributos), columns=preprocessador.get_feature_names_out())

# Mostrando df_atributos pré-processado
print("df_atributos pré-processado")
display(df_atributos_pre.head())

df_atributos pré-processado


,Você participa da comunidade LGBTQIA+,Idade,Gênero,Orientação Sexual,Localização,Nível de Escolaridade,Você usa aplicativos para procurar oportunidades de emprego?,Preferência por Cursos,Você já sentiu desafios para entrar no mercado de trabalho devido ao seu gênero,Você tem interesse em empreender?,Qual é a sua situação diante ao mercado de trabalho?,"Você usa redes sociais (exemplo: X, Facebook, Instagram, Youtube)"
0,0.0,5.0,3.0,3.0,3.0,0.0,1.0,3.0,1.0,0.0,1.0,1.0
1,0.0,5.0,3.0,3.0,3.0,0.0,2.0,1.0,1.0,0.0,1.0,1.0
2,0.0,5.0,3.0,5.0,2.0,0.0,0.0,1.0,0.0,2.0,1.0,1.0
3,0.0,5.0,3.0,1.0,3.0,0.0,0.0,1.0,1.0,0.0,1.0,1.0
4,0.0,5.0,2.0,6.0,3.0,6.0,0.0,0.0,1.0,1.0,0.0,1.0


In [36]:
#Resposta
le = LabelEncoder()

# Transformando dados da variável resposta em dados quantitativos
df_resposta = pd.DataFrame(le.fit_transform(df_resposta))

# Transformando df de volta para o formato Data
df_resposta_pre = pd.DataFrame(le.fit_transform(df_resposta))

# Mostrando df_resposta pré-processado
print("df_resposta pré-processado")
display(df_resposta_pre.head())

df_resposta pré-processado


C:\Users\oliviadomingues-ieg\AppData\Local\Temp\ipykernel_10004\2548990530.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_atributos.rename(columns={v: v.strip()}, inplace=True)
C:\Users\oliviadomingues-ieg\AppData\Local\Temp\ipykernel_10004\2548990530.py:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_atributos[df_atributos.columns[k]] = le.fit_transform(df_atributos[df_atributos.columns[k]])
C:\Users\oliviadomingues-ieg\AppData\Local\Temp\ipykernel_10004\2548990530.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a

,0
0,4
1,3
2,2
3,3
4,2
